In [2]:
import glob

import numpy as np
import pandas as pd

# import seaborn as sns
# import matplotlib.pyplot as plt
# import matplotlib.colors as mcolors

# import plotly.express as px
# import plotly.graph_objects as go

In [23]:
columns = ['io','method','processes','size','duration']
results = pd.read_csv('../results/io_results.csv', names=columns)

In [24]:
results

,io,method,processes,size,duration
0,write,serial,1,1000,0
1,read,serial,1,1000,0
2,write,serial,1,10000,0
3,read,serial,1,10000,0
4,write,serial,1,100000,0
5,read,serial,1,100000,0
6,write,serial,1,1000000,0
7,read,serial,1,1000000,0
8,write,serial,1,10000000,0
9,read,serial,1,10000000,0


In [26]:
results.duration.min()

0

In [21]:
count=1000000
x = np.fromfile(
    file=f'../data/serial/unsorted_{count}.bin',
    dtype=np.float64,
    count=90000002
)

In [22]:
x.shape

(1000000,)